<a href="https://colab.research.google.com/github/MtTake/PINNs/blob/main/2D_Navier_Eq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 弾性力学の支配方程式

- つり合い式 \\
- 適合条件式(変位-ひずみ関係式)
- 構成式(応力-ひずみ関係式) \\

### 1. つり合い式
　二次元で考えると、

$$
\frac{\partial \sigma_x}{\partial x} + \frac{\partial \tau_{xy}}{\partial y} + X = 0 \\
\frac{\partial \sigma_y}{\partial y} + \frac{\partial \tau_{xy}}{\partial x} + Y = 0 \\
 $$

### 2. 適合条件式

$$
\frac{\partial^2 \epsilon_x}{\partial y^2} + \frac{\partial^2 \epsilon_y}{\partial x^2} = \frac{\partial^2 \gamma_{xy}}{\partial x \partial y} \\
\epsilon_x = \frac{\partial u}{\partial x}, \quad
\epsilon_y = \frac{\partial v}{\partial y}, \quad
\gamma_{xy} = \frac{\partial u}{\partial y} + \frac{\partial v}{\partial x}
$$

### 3. 構成式

$$
\begin {Bmatrix}
       \epsilon_x \\
       \epsilon_y \\
       \epsilon_z \\
       \gamma_{xy} \\
       \gamma_{yz} \\
       \gamma_{zx}
\end {Bmatrix}
= \frac{1}{E}
\begin {bmatrix}
       1 & -v & -v & 0 & 0 & 0 \\
       -v & 1 & -v & 0 & 0 & 0 \\
       -v & -v & 1 & 0 & 0 & 0 \\
       0 & 0 & 0 & 2(1+v) & 0 & 0\\
       0 & 0 & 0 & 0 & 2(1+v) & 0 \\
       0 & 0 & 0 & 0 & 0 & 2(1+v)
\end {bmatrix}
\begin {Bmatrix}
       \sigma_x \\
       \sigma_y \\
       \sigma_z \\
       \tau_{xy} \\
       \tau_{yz} \\
       \tau_{zx}
\end {Bmatrix}
$$

# Navier の方程式

$$
G \left(\nabla^2u_i + \frac{1}{1-2\nu}\frac{\partial \epsilon_i}{\partial x_i}
 + X_i \right) = \rho \frac{\partial^2 u_i}{\partial t^2} \quad (i=1,2,3)
$$

静的釣合い条件では、$i$ を $x,y,z$ で書くと、

# 損失関数

損失関数は、ニューラルネットワークの出力が連続体方程式を満たすようにする．具体的には、以下のような損失関数を定義：

$$ L_{\text{total}} = L_{\text{PDE}} + L_{\text{BC}} + L_{\text{IC}}. $$

- **PDE損失項** $L_{\text{PDE}}$：ネットワークの出力がPDEを満たすようにする損失です。具体的には、次の手順で計算します：

  1. ネットワークの出力 $u(x,t$) から、PDEに必要な偏微分を計算します（例えば、$\frac{\partial u}{\partial t}$ や $\frac{\partial^2 u}{\partial x^2}$）。
  2. PDEの左辺 $\mathcal{L}(u(x,t)$) を計算し、その結果がゼロに近くなるようにします。
  3. 損失として、PDEの残差（$\mathcal{L}(u(x,t))$）の二乗和を使用します。

  例えば、熱方程式の場合のPDE損失は次のように書けます：

  $$ L_{\text{PDE}} = \frac{1}{N_{\text{PDE}}} \sum_{i=1}^{N_{\text{PDE}}} \left(\frac{\partial u(x_i, t_i)}{\partial t} - \alpha \frac{\partial^2 u(x_i, t_i)}{\partial x^2} \right)^2, $$

  ここで、$(x_i, t_i)$ はランダムにサンプリングされたポイントです。

- **境界条件損失項** $L_{\text{BC}}$：境界条件（例えば、空間の端点での値やフラックス条件）を満たすようにする損失です。

  例えば、$x = 0$ と $x = L$ での境界条件が $u(0, t) = u_L$ と $u(L, t) = u_R$ である場合、次のように計算します：

  $$ L_{\text{BC}} = \frac{1}{N_{\text{BC}}} \sum_{j=1}^{N_{\text{BC}}} \left(u(x_j, t_j) - u_{\text{BC}}(x_j, t_j) \right)^2, $$

  ここで、$u_{\text{BC}}$ は境界条件に従った値です。

- **初期条件損失項** $L_{\text{IC}}$：初期条件（例えば、$t=0$ のときの $u(x,0)$ の値）を満たすようにする損失です。初期条件損失は次のように書けます：

  $$ L_{\text{IC}} = \frac{1}{N_{\text{IC}}} \sum_{k=1}^{N_{\text{IC}}} \left(u(x_k, 0) - u_{\text{IC}}(x_k) \right)^2, $$

  ここで、$u_{\text{IC}}$ は初期条件に従った値です。

# 学習

PINNsのトレーニングは、損失関数 $L_{\text{total}}$ を最小化することによって行われます。通常、確率的勾配降下法（SGD）やそのバリエーション（例えばAdam）を使用します。トレーニングデータには、ランダムにサンプリングした空間座標と時間座標が含まれ、これらに対してネットワークの出力がPDE、境界条件、初期条件を満たすように調整されます。

これが連続体方程式をPINNsの損失関数に組み込む基本的な流れです。具体的な実装には、数値微分、サンプリング戦略、ネットワークのアーキテクチャなどの詳細な調整が必要です。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
num_points = 100

n_in_out = 2
n_make_out = 6


In [ ]:
# ニューラルネットワークの定義
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.fc1 = nn.Linear(n_in_out, 32)
        self.fc2 = nn.Linear(32, 64)
        # self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 256)
        # self.bn4 = nn.BatchNorm1d(256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, n_make_out)
        self.fc8 = nn.Sigmoid()
        self.apply(self.init_weights) # 重み初期化を適用

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)  # Xavier初期化
            if m.bias is not None:
                init.zeros_(m.bias)  # バイアスはゼロ初期化

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        # x = F.leaky_relu(self.bn2(self.fc2(x)), negative_slope=0.01)
        x = F.leaky_relu(self.fc2(x), negative_slope=0.01)
        x = F.leaky_relu(self.fc3(x), negative_slope=0.01)
        # x = F.leaky_relu(self.bn4(self.fc4(x)), negative_slope=0.01)
        x = F.leaky_relu(self.fc4(x), negative_slope=0.01)
        x = F.leaky_relu(self.fc5(x), negative_slope=0.01)
        x = F.leaky_relu(self.fc6(x), negative_slope=0.01)
        x = F.leaky_relu(self.fc7(x), negative_slope=0.01)
        x = self.fc8(x)
        return x

In [ ]:
# Navier方程式の計算
def derivatives(u, e, f, x):
    u_x = torch.autograd.grad(outputs=u, inputs=x, grad_outputs=torch.ones_like(u),
                              create_graph=True, retain_graph=True)[0]
    u_xx = torch.autograd.grad(outputs=u_x[:, 0], inputs=x, grad_outputs=torch.ones_like(u_x[:, 0]),
                               create_graph=True, retain_graph=True)[0][:, 0]
    u_yy = torch.autograd.grad(outputs=u_x[:, 1], inputs=x, grad_outputs=torch.ones_like(u_x[:, 1]),
                               create_graph=True, retain_graph=True)[0][:, 1]
    u_e = torch.autograd.grad(outputs=e, inputs=x, grad_outputs=torch.ones_like(e),
                              create_graph=True, retain_graph=True)[0]

    return u_xx + u_yy + 1/(1-0.3)*(u_e.sum(dim=1) + f.sum(dim=1))

In [ ]:
# 損失関数の定義
def loss_function(model, x_bc, u_bc, x_ic, f_ic):

    # 境界条件損失
    y_bc = torch.rand((2 * N_bc, 1)) * L                # y座標はランダム
    bc_pred = model(torch.cat((x_bc, y_bc), dim=1))     # MLPを通す
    u_bc_pred = bc_pred[:, :2]                          # 変位を抽出
    loss_bc = torch.mean((u_bc_pred - u_bc)**2)

    # 初期条件損失
    ic_pred = model(x_ic)                               # MLPを通す
    f_ic_pred = ic_pred[:, 4:6]                         # 荷重を抽出
    loss_ic = torch.mean((f_ic_pred - f_ic)**2)

    return loss_bc + loss_ic

In [ ]:
# 訓練データの生成
def generate_data(num_points):
    # x と t の範囲を設定
    x_plot = torch.linspace(-L, L, num_points).reshape(-1, 1) # [100, 1]
    y_plot = torch.linspace(-L, L, num_points).reshape(-1, 1) # [100, 1]

    # メッシュグリッドを作成
    # X_g は x_plot の値を列方向に繰り返した行列
    # Y_g は y_plot の値を行方向に繰り返した行列
    # X_g=[100, 100],Y_g=[100, 100]
    X_g, Y_g = torch.meshgrid(x_plot.squeeze(), y_plot.squeeze(), indexing='ij')

    # Flatten 行優先で1行目のあとに2行目を並べる
    # cat dim=1 で全組み合わせ (n,2)
    X_flat = X_g.reshape(-1, 1) # [10000, 1]
    Y_flat = Y_g.reshape(-1, 1) # [10000, 1]

    # gen_data = torch.rand(num_points, 2) * 2 - 1  # [-1, 1]の範囲でサンプルを生成
    gen_data = torch.cat((X_flat, Y_flat), dim=1)

    return gen_data

# ハイパーパラメータ
num_epochs = 10000
learning_rate = 0.001

L = 1.0  # 空間範囲
FX = 1.0  # 荷重
N_bc = 100
N_ic = 100

# モデルの初期化
model = PINN()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# ***** test *******************************************************************
num_points=100                  # サンプル点数
x = generate_data(num_points)   # サンプル点を生成
x.requires_grad = True          # 勾配計算を可能にする
u = model(x)                    # モデルの出力を計算

print(x.shape)
print('x=',x[:5])
print(u.shape)
print('u=',u[:5])

re_ux = u[:, :2]                 # 配列を2次元に分離
re_ex = u[:, 2:4]
re_fx = u[:, 4:6]

print('*** Output ***')
print(re_ux.shape)
print('re_ux=',re_ux[:5])
print(re_ex.shape)
print('re_ex=',re_ex[:5])
print(re_fx.shape)
print('re_fx=',re_fx[:5])

# Navier方程式計算
u_x = torch.autograd.grad(outputs=re_ux, inputs=x, grad_outputs=torch.ones_like(re_ux),
                          create_graph=True, retain_graph=True)[0]
# grad_outputs=torch.ones_like(u_x[:, 0])を指定するとxの最初の次元に関する勾配を計算
u_xx = torch.autograd.grad(outputs=u_x[:, 0], inputs=x, grad_outputs=torch.ones_like(u_x[:, 0]),
                            create_graph=True, retain_graph=True)[0][:, 0]
u_yy = torch.autograd.grad(outputs=u_x[:, 1], inputs=x, grad_outputs=torch.ones_like(u_x[:, 1]),
                            create_graph=True, retain_graph=True)[0][:, 1]
u_e = torch.autograd.grad(outputs=re_ex, inputs=x, grad_outputs=torch.ones_like(re_ex),
                          create_graph=True, retain_graph=True)[0]

print('*** Derivatives ***')
print(u_x.shape)
print('u_x=',u_x[:5])
print(u_xx.shape)
print('u_xx=',u_xx[:5])
print(u_yy.shape)
print('u_yy=',u_yy[:5])
print(u_e.shape)
print('u_e=',u_e[:5])
print('u_e.sum=', u_e.sum(dim=1)[:5])
print('u_xx + u_yy + 1/(1-2*0.3)*u_e.sum(dim=1) + re_fx.sum(dim=1)=', (u_xx + u_yy + 1/(1-2*0.3)*(u_e.sum(dim=1)) + (re_fx.sum(dim=1)))[:5])

print('================================================')
# 入力テンソル b を作成（勾配追跡を有効にする）
b = torch.tensor(np.array([[-0.4506,  0.7401], [-0.1686, -0.2135]]), requires_grad=True)

# 出力テンソル a を適当な計算から得る（ここでは b の2乗を例にする）
a = b ** 2  # これは勾配を追跡する

# 勾配計算
grad = torch.autograd.grad(outputs=a, inputs=b, grad_outputs=torch.ones_like(a),
                           create_graph=True, retain_graph=True)[0]

print('grad sample =', grad)  # 結果を表示

In [ ]:
x_lwr = torch.full((3, 1), -1)
x_upr = torch.full((3, 1), 1)
y_rnd = torch.rand((2*3, 1)) * 1

x_0 = torch.cat((x_lwr, x_upr), dim=0)
ux_0 = torch.zeros((2 * 3, 1), dtype=torch.float32)
print(x_0)
print(torch.cat((x_0, y_rnd), dim=1))

In [ ]:
# 訓練ループ
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # サンプル点を生成
    x = generate_data(num_points)
    x.requires_grad = True  # 勾配計算を可能にする

    # モデルの出力を計算
    u = model(x)

    # デコーダからの出力を3次元の座標情報、変位情報と荷重情報に分ける
    re_ux = u[:, :2]                 # 配列を2次元に分離
    re_ex = u[:, 2:4]
    re_fx = u[:, 4:6]

    # Navierの残差を計算
    derivatives_residual = derivatives(re_ux, re_ex, re_fx, x)

    # 損失を計算
    loss_nav = torch.mean(derivatives_residual**2)

    # 境界条件と初期条件のサンプル
    x_bc = torch.cat((torch.full((N_bc, 1), -L), torch.full((N_bc, 1), L)), dim=0) # x=-1,x=1の位置の配列
    u_bc = torch.zeros((2 * N_bc, 1), dtype=torch.float32)                         # x=-1,x=1の境界条件の配列

    x_ic = torch.tensor([[0, 0]], dtype=torch.float32)                             # 荷重点を原点に設定
    f_ic = torch.tensor([[1, 1]], dtype=torch.float32)                             # 荷重をfx=1,fy=1で入れる

    loss_cnd = loss_function(model, x_bc, u_bc, x_ic, f_ic)

    loss = loss_nav + loss_cnd

    # 逆伝播と最適化
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
# 結果の可視化
def plot_solution(net, x, y):
    with torch.no_grad():
        net.eval()
        # ネットワークの出力を計算
        # 全組み合わせ[10000, 2]で、出力[10000, 6]を得る
        u = net(torch.cat((x, y), dim=1)).numpy()
        ux = np.sqrt(u[:, :1]**2 + u[:, 1:2]**2)

        # x と t を 1D 配列に変換 ⇒ x= (10000,),t= (10000,)
        x = x.numpy().flatten()
        y = y.numpy().flatten()

        # メッシュグリッドの生成
        # 再度、[100, 100]にする、x,tをダブり抜きしてからmeshgrid
        X, Y = np.meshgrid(np.unique(x), np.unique(y))

        # u を 2D 配列に変換 [10000, 1]⇒[100, 100]
        U = ux.reshape(len(np.unique(y)), len(np.unique(x)))

        # プロット
        plt.figure(figsize=(10, 6))
        contour = plt.contourf(X, Y, U, cmap='viridis')
        plt.colorbar(contour)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Predicted Solution')
        plt.show()

# x と t の範囲を設定
x_plot = torch.linspace(-L, L, 100).reshape(-1, 1) # [100, 1]
y_plot = torch.linspace(-L, L, 100).reshape(-1, 1) # [100, 1]

# メッシュグリッドを作成
# X_g は x_plot の値を列方向に繰り返した行列
# Y_g は y_plot の値を行方向に繰り返した行列
# X_g=[100, 100],Y_g=[100, 100]
X_g, Y_g = torch.meshgrid(x_plot.squeeze(), y_plot.squeeze(), indexing='ij')

# Flatten 行優先で1行目のあとに2行目を並べる
# cat dim=1 で全組み合わせ (n,2)
X_flat = X_g.reshape(-1, 1) # [10000, 1]
Y_flat = Y_g.reshape(-1, 1) # [10000, 1]

# 結果をプロット
plot_solution(model, X_flat, Y_flat)
